# [Issue# 1897 Additional Visuals for PUC Analysis](https://github.com/cal-itp/data-analyses/issues/1897)

Received list of transit operators cohorts that may be exempt from efficiency reporting, per PUC 99314.11, .6 and .7. 
- create visuals based on grouping set by list
- recreate visuals based on previous notebook work

## [99314.6](https://leginfo.legislature.ca.gov/faces/codes_displaySection.xhtml?sectionNum=99314.6.&lawCode=PUC)
>`funds shall be allocated for operating or capital purpose` pursuant to Sections 99313 and 99314 to an operator `if the operator meets either of the following efficiency standards`:
>- (A) `The operator shall receive its entire allocation`, and any or all of this allocation may be used for operating purposes, if the operator’s `total operating cost per revenue vehicle hour` in the latest year for which audited data are available `does not exceed the sum of the preceding year’s total operating cost per revenue vehicle hour and an amount equal to the product of the percentage change in the Consumer Price Index for the same period multiplied by the preceding year’s total operating cost per revenue vehicle hour.`
>- (B) The operator shall receive its entire allocation, and any or all of this allocation may be used for operating purposes, `if the operator’s average total operating cost per revenue vehicle hour` in the latest three years for which audited data are available `does not exceed the sum of the average of the total operating cost per revenue vehicle hour in the three years preceding the latest year for which audited data are available and an amount equal to the product of the average percentage change in the Consumer Price Index for the same period multiplied by the average total operating cost per revenue vehicle hour in the same three years`.
## [99314.7 (mainly MTC specific)](https://leginfo.legislature.ca.gov/faces/codes_displaySection.xhtml?lawCode=PUC&sectionNum=99314.7.)
>the `Metropolitan Transportation Commission` shall apply the following eligibility standards to the operators within the region subject to its jurisdiction:

# [99314.11](https://leginfo.legislature.ca.gov/faces/codes_displaySection.xhtml?sectionNum=99314.11.&nodeTreePath=17.11.2.8&lawCode=PUC)
>`Sections 99314.6 and 99314.7 do not apply to an operator for a fiscal year in which the operator expended from local funding an amount for transit operations not less than the amount the operator expended from local funding for transit operations during the 2018–19 fiscal year.` As used in this subdivision, “local funding” means any nonstate grant funds or other revenues generated by, earned by, or distributed to, an operator.

Meaning, if a transit operator spent local funds >= the local funds spent during FY 2018-2019, they are exempt from meeting efficiency standards(?)

In [107]:
import pandas as pd
import altair as alt
from functools import cache
from calitp_data_analysis.gcs_pandas import GCSPandas
from calitp_data_analysis.sql import get_engine, to_snakecase, query_sql

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

@cache
def gcs_pandas():
    return GCSPandas()

# Read in cohort list data

In [2]:
cohort_data = gcs_pandas().read_csv("gs://calitp-analytics-data/data-analyses/ntd/fbr_local_funding_by_cohorts_2019-2024_compiled.csv")

In [34]:
cohort_data.columns = cohort_data.columns.str.lower()
cohort_data["ntd_id"] = cohort_data["ntd_id"].astype("str")

In [35]:
display(
    cohort_data.info(),
    cohort_data.head()
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1797 entries, 0 to 1796
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ntd_id       1797 non-null   object
 1   urban_rural  1797 non-null   object
 2   cohort       1797 non-null   object
 3   metric       1797 non-null   object
 4   year         1797 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 70.3+ KB


None

,ntd_id,urban_rural,cohort,metric,year
0,91018,Rural,Group A,Farebox Recovery Ratio,2019
1,91062,Rural,Group A,Farebox Recovery Ratio,2019
2,91088,Rural,Group A,Farebox Recovery Ratio,2019
3,91043,Rural,Group A,Farebox Recovery Ratio,2019
4,91036,Rural,Group A,Farebox Recovery Ratio,2019


In [5]:
cohort_data.value_counts(
    subset=["urban_rural","metric","cohort","year"]
)

urban_rural  metric                          cohort   year
Urban        Farebox Recovery Ratio          Group B  2023    52
                                                      2024    52
                                                      2019    51
                                                      2020    51
             Local Funding % Change vs 2019  Group B  2024    50
                                                              ..
Rural        Farebox Recovery Ratio          Group A  2023    12
                                                      2021    12
                                             Group C  2024    12
                                             Group A  2024    11
                                                      2022    11
Name: count, Length: 72, dtype: int64

# Read in analysis data from prev notebook

In [6]:
gcs_path = "gs://calitp-analytics-data/data-analyses/ntd/"
ntd_name = "puc_analysis_data.parquet"
ntd_analysis_data = gcs_pandas().read_parquet(f"{gcs_path}{ntd_name}")

In [7]:
ntd_analysis_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4152 entries, 0 to 4151
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   ntd_id             4002 non-null   object  
 1   source_agency      4152 non-null   object  
 2   agency_status      4152 non-null   object  
 3   primary_uza_name   4152 non-null   object  
 4   uza_population     4152 non-null   int64   
 5   uza_area_sq_miles  4152 non-null   float64 
 6   year               4152 non-null   int64   
 7   mode               4152 non-null   object  
 8   service            4152 non-null   object  
 9   reporter_type      4152 non-null   object  
 10  total_vrh          2623 non-null   float64 
 11  total_upt          2623 non-null   float64 
 12  total_pmt          2623 non-null   float64 
 13  ntd_id_2022        3798 non-null   object  
 14  county             3798 non-null   object  
 15  caltrans_district  3798 non-null   object  
 16  _merge     

In [8]:
ntd_analysis_data["year"].unique()

array([2018, 2019, 2020, 2021, 2022, 2023])

# May need to requery this data to include 2024
is 2024 NTD data in the warehouse now? copy pasted from initial puc analysis notebook.

In [10]:
metric_list = [
    "pmt",
    "upt",
    "vrh",
    # "opexp_total" # not needed for this project
]

# empty list for appending DFs
df_list = []

# loop to query pmt, upt and vrh from 2018 to 2024
for metric in metric_list:
        query = f"""
        SELECT
          ntd_id,
          source_agency,
          agency_status,
          primary_uza_name,
          uza_population,
          uza_area_sq_miles,
          year,
          mode,
          type_of_service,
          reporter_type,
          SUM({metric}) AS total_{metric},
        FROM
          `cal-itp-data-infra.mart_ntd_funding_and_expenses.fct_service_data_and_operating_expenses_time_series_by_mode_{metric}`
        WHERE
          source_state = "CA"
          AND year BETWEEN 2018 AND 2024
        GROUP BY
          ntd_id,
          source_agency,
          agency_status,
          primary_uza_name,
          uza_population,
          uza_area_sq_miles,
          year,
          mode,
          type_of_service,
          reporter_type
        """
        # create df
        metric = query_sql(query, as_df=True)

        # append df to list
        df_list.append(metric)

# unpack list into separate DFs
ntd_pmt, ntd_upt, ntd_vrh = df_list

In [21]:
display( 
    ntd_upt.head(3)
)

,ntd_id,source_agency,agency_status,primary_uza_name,uza_population,uza_area_sq_miles,year,mode,type_of_service,reporter_type,total_upt
0,90003,San Francisco Bay Area Rapid Transit District ...,Active,"San Francisco--Oakland, CA",3515933,513.8,2019,MG,PT,Full Reporter,886515.0
1,90003,San Francisco Bay Area Rapid Transit District ...,Active,"San Francisco--Oakland, CA",3515933,513.8,2018,MB,PT,Full Reporter,NaN
2,90003,San Francisco Bay Area Rapid Transit District ...,Active,"San Francisco--Oakland, CA",3515933,513.8,2020,MB,PT,Full Reporter,NaN


## merge all the metrics together

In [22]:
merge_on_col = [
    "ntd_id",
    "year",
    "source_agency",
    "agency_status",
    "primary_uza_name",
    "uza_population",
    "uza_area_sq_miles",
    "mode",
    "type_of_service",
    "reporter_type",
]

merge_1 = ntd_vrh.merge(ntd_upt, on=merge_on_col, how="inner")
# merge_2 = merge_1.merge(ntd_vrh, on=merge_on_col, how = "inner")

ntd_metrics_merge = merge_1.merge(ntd_pmt, on=merge_on_col, how="inner")

In [23]:
ntd_metrics_merge.head(3)

,ntd_id,source_agency,agency_status,primary_uza_name,uza_population,uza_area_sq_miles,year,mode,type_of_service,reporter_type,total_vrh,total_upt,total_pmt
0,90003,San Francisco Bay Area Rapid Transit District ...,Active,"San Francisco--Oakland, CA",3515933,513.8,2019,MG,PT,Full Reporter,19815.0,886515.0,2819118.0
1,90003,San Francisco Bay Area Rapid Transit District ...,Active,"San Francisco--Oakland, CA",3515933,513.8,2019,MB,PT,Full Reporter,NaN,NaN,NaN
2,90003,San Francisco Bay Area Rapid Transit District ...,Active,"San Francisco--Oakland, CA",3515933,513.8,2021,MG,PT,Full Reporter,17819.0,112981.0,359280.0


## get districts for ntd ID
- Do i still need district data for this specific analysis?

In [24]:
for metric in metric_list:
        query = f"""
        SELECT
          `mart_transit_database.dim_organizations`.`key` AS `key`,
          `mart_transit_database.dim_organizations`.`source_record_id` AS `source_record_id`,
          `mart_transit_database.dim_organizations`.`name` AS `name`,
          `mart_transit_database.dim_organizations`.`ntd_id_2022` AS `ntd_id_2022`,
          `Bridge_Organizations_X_Headquarters_County_Geography___Key`.`county_geography_name` AS `county`,
          `Dim_County_Geography___County_Geography_Key`.`caltrans_district` AS `caltrans_district`
        FROM
          `mart_transit_database.dim_organizations`

        LEFT JOIN `mart_transit_database.bridge_organizations_x_headquarters_county_geography` AS `Bridge_Organizations_X_Headquarters_County_Geography___Key` ON `mart_transit_database.dim_organizations`.`key` = `Bridge_Organizations_X_Headquarters_County_Geography___Key`.`organization_key`
          LEFT JOIN `mart_transit_database.dim_county_geography` AS `Dim_County_Geography___County_Geography_Key` ON `Bridge_Organizations_X_Headquarters_County_Geography___Key`.`county_geography_key` = `Dim_County_Geography___County_Geography_Key`.`key`
        WHERE
          (
            `mart_transit_database.dim_organizations`.`_is_current` = TRUE
          )

           AND (
            `mart_transit_database.dim_organizations`.`ntd_id_2022` IS NOT NULL
          )
          AND (
            (
              `mart_transit_database.dim_organizations`.`ntd_id_2022` <> ''
            )

            OR (
              `mart_transit_database.dim_organizations`.`ntd_id_2022` IS NULL
            )
          )
          AND (
            `Bridge_Organizations_X_Headquarters_County_Geography___Key`.`_is_current` = TRUE
          )
          AND (
            `Dim_County_Geography___County_Geography_Key`.`_is_current` = TRUE
          )
        """
        # create df
        ntd_id_x_district = query_sql(query, as_df=True)
        
ntd_id_x_district["caltrans_district"] = ntd_id_x_district["caltrans_district"].astype("str")

In [25]:
ntd_id_x_district.head()

,key,source_record_id,name,ntd_id_2022,county,caltrans_district
0,d84a961daa618c733f9d9c3bd49c322f,recJtH0Ae8YNo01aj,Access Services,90157,Los Angeles,7
1,9b5971d16d58e4fcafa694ee7fa33b12,rec79AM4tMwdokWhE,Alpine County,91116,Alpine,10
2,e5de5083d68e8c2463a784ceb13e91f2,recUTSH4TT1wB3RSC,Attentive Transportation LLC,90314,Sacramento,3
3,957618c89db2f5e992caa5ca2e6086ab,rec7F6JKLMVrRhQJU,Bishop Paiute Tribe,99268,Inyo,9
4,a024fabd0002f9c9bd636042de30715d,recE6qJFuoREa9EHg,Calaveras County,91063,Calaveras,10


## merge the ntd metrics with Caltrans Districts

In [26]:
ntd_metrics_merge = ntd_metrics_merge.merge(
    ntd_id_x_district[["ntd_id_2022","county","caltrans_district"]],
    left_on = "ntd_id",
    right_on = "ntd_id_2022",
    how="inner",
    indicator=True
)

In [28]:
ntd_metrics_merge.head()

,ntd_id,source_agency,agency_status,primary_uza_name,uza_population,uza_area_sq_miles,year,mode,type_of_service,reporter_type,total_vrh,total_upt,total_pmt,ntd_id_2022,county,caltrans_district,_merge
0,90003,San Francisco Bay Area Rapid Transit District ...,Active,"San Francisco--Oakland, CA",3515933,513.8,2019,MG,PT,Full Reporter,19815.0,886515.0,2819118.0,90003,San Francisco,4,both
1,90003,San Francisco Bay Area Rapid Transit District ...,Active,"San Francisco--Oakland, CA",3515933,513.8,2019,MB,PT,Full Reporter,NaN,NaN,NaN,90003,San Francisco,4,both
2,90003,San Francisco Bay Area Rapid Transit District ...,Active,"San Francisco--Oakland, CA",3515933,513.8,2021,MG,PT,Full Reporter,17819.0,112981.0,359280.0,90003,San Francisco,4,both
3,90003,San Francisco Bay Area Rapid Transit District ...,Active,"San Francisco--Oakland, CA",3515933,513.8,2021,DR,PT,Full Reporter,NaN,NaN,NaN,90003,San Francisco,4,both
4,90003,San Francisco Bay Area Rapid Transit District ...,Active,"San Francisco--Oakland, CA",3515933,513.8,2019,YR,DO,Full Reporter,41924.0,2225056.0,15283299.0,90003,San Francisco,4,both


# merge ntd metrics with cohort data
- merge on ntd_id
- are there any unmerged rows?

In [33]:
cohort_data.dtypes

ntd_id          int64
urban_rural    object
cohort         object
metric         object
year            int64
dtype: object

In [32]:
ntd_metrics_merge.dtypes

ntd_id                 object
source_agency          object
agency_status          object
primary_uza_name       object
uza_population          int64
uza_area_sq_miles     float64
year                    int64
mode                   object
type_of_service        object
reporter_type          object
total_vrh             float64
total_upt             float64
total_pmt             float64
ntd_id_2022            object
county                 object
caltrans_district      object
_merge               category
dtype: object

In [49]:
ntd_cohort_merge = ntd_metrics_merge.drop(columns="_merge").merge(
    cohort_data,
    left_on = ["ntd_id","year"],
    right_on = ["ntd_id","year"],
    indicator= True,
)

In [50]:
ntd_cohort_merge.head()

,ntd_id,source_agency,agency_status,primary_uza_name,uza_population,uza_area_sq_miles,year,mode,type_of_service,reporter_type,total_vrh,total_upt,total_pmt,ntd_id_2022,county,caltrans_district,urban_rural,cohort,metric,_merge
0,90003,San Francisco Bay Area Rapid Transit District ...,Active,"San Francisco--Oakland, CA",3515933,513.8,2019,MG,PT,Full Reporter,19815.0,886515.0,2819118.0,90003,San Francisco,4,Urban,Group A,Farebox Recovery Ratio,both
1,90003,San Francisco Bay Area Rapid Transit District ...,Active,"San Francisco--Oakland, CA",3515933,513.8,2019,MG,PT,Full Reporter,19815.0,886515.0,2819118.0,90003,San Francisco,4,Urban,Group B,Local Funding % Change vs 2019,both
2,90003,San Francisco Bay Area Rapid Transit District ...,Active,"San Francisco--Oakland, CA",3515933,513.8,2019,MB,PT,Full Reporter,NaN,NaN,NaN,90003,San Francisco,4,Urban,Group A,Farebox Recovery Ratio,both
3,90003,San Francisco Bay Area Rapid Transit District ...,Active,"San Francisco--Oakland, CA",3515933,513.8,2019,MB,PT,Full Reporter,NaN,NaN,NaN,90003,San Francisco,4,Urban,Group B,Local Funding % Change vs 2019,both
4,90003,San Francisco Bay Area Rapid Transit District ...,Active,"San Francisco--Oakland, CA",3515933,513.8,2021,MG,PT,Full Reporter,17819.0,112981.0,359280.0,90003,San Francisco,4,Urban,Group B,Farebox Recovery Ratio,both


In [71]:
ntd_metrics_merge.shape

(3801, 17)

In [51]:
# any unmerged rows? NONE
ntd_cohort_merge["_merge"].value_counts()

_merge
both          4965
left_only        0
right_only       0
Name: count, dtype: int64

In [74]:
ntd_cohort_merge["metric"].unique()

array(['Farebox Recovery Ratio', 'Local Funding % Change vs 2019'],
      dtype=object)

In [ ]:
# Sanity check
# pick up a couple of NTD ID, see if the merge data tracks with the cohort data
sample_ids = ntd_cohort_merge["ntd_id"].sample(3).to_list()
keep_cols=[
    "ntd_id",
    "source_agency",
    "mode",
    "type_of_service",
    "total_vrh",
    "total_pmt",
    "total_upt",
    "urban_rural",
    "cohort",
    "metric",
    "year"
]

for sample_id in sample_ids:
    display(
        f"Sameple NTD ID: {sample_id}",
        "cohort data",
        cohort_data[
            (cohort_data["ntd_id"]== sample_id)
            & (cohort_data["year"].isin([2023,2024]))
            ].sort_values(by=["urban_rural","cohort","metric","year"]).head(5),
        "merge table",
        ntd_cohort_merge[
            (ntd_cohort_merge["ntd_id"]== sample_id)
            & (ntd_cohort_merge["year"].isin([2023,2024]))
            ][keep_cols].sort_values(by=["urban_rural","cohort","metric","year"]),
        
    )

# cohort data matches, 
# looks a little weird since the ntd metrics is per mode and TOS. the cohort data becomes categorical. GTG

# Save merged cohort data

In [99]:
cort_merge_filname = "ntd_cohort_data_2026-01-26.parquet"
# gcs_pandas().data_frame_to_parquet(ntd_cohort_merge,f"{gcs_path}{cort_merge_filname}")

# separate list by both metrics (farebox and funding change)

In [91]:
cohort_merge_farebox = ntd_cohort_merge[ntd_cohort_merge["metric"]=="Farebox Recovery Ratio"]
cohort_merge_funding = ntd_cohort_merge[ntd_cohort_merge["metric"]=="Local Funding % Change vs 2019"]

In [101]:
display(
    cohort_merge_farebox.shape,
    cohort_merge_funding.shape,
    cohort_merge_farebox["metric"].unique(),
    cohort_merge_funding["metric"].unique(),
    cohort_merge_farebox.columns
)

(2460, 20)

(2505, 20)

array(['Farebox Recovery Ratio'], dtype=object)

array(['Local Funding % Change vs 2019'], dtype=object)

Index(['ntd_id', 'source_agency', 'agency_status', 'primary_uza_name',
       'uza_population', 'uza_area_sq_miles', 'year', 'mode',
       'type_of_service', 'reporter_type', 'total_vrh', 'total_upt',
       'total_pmt', 'ntd_id_2022', 'county', 'caltrans_district',
       'urban_rural', 'cohort', 'metric', '_merge'],
      dtype='object')

# Group aggregation

## melt big DF 
- so all columns are under 1 column.

In [111]:
group_list_melt = [
    "source_agency",
    "year",
    "ntd_id",
    "caltrans_district",
    "mode",
    "type_of_service",
    "urban_rural",
    "cohort",
    "metric"
]

value_cols = ["total_upt", "total_vrh", "total_pmt"]

melt_farebox = pd.melt(
    cohort_merge_farebox,
    id_vars=group_list_melt,
    value_vars=value_cols,
    var_name="ntd_metric",
    value_name="ntd_metric_value",
    ignore_index=True,
)

melt_funding = pd.melt(
    cohort_merge_funding,
    id_vars=group_list_melt,
    value_vars=value_cols,
    var_name="ntd_metric",
    value_name="ntd_metric_value",
    ignore_index=True,
)

In [113]:
display(
    melt_farebox.shape,
    melt_funding.shape
)

(7380, 11)

(7515, 11)

In [110]:
melt_farebox[melt_farebox["ntd_id"].isin([sample_ids[1]])].sort_values(by=["year","mode","type_of_service"])

,source_agency,year,ntd_id,reporter_type,caltrans_district,mode,type_of_service,urban_rural,cohort,metric,ntd_metric,ntd_metric_value
618,The Eastern Contra Costa Transit Authority,2019,90162,Full Reporter,4,DR,PT,Urban,Group C,Farebox Recovery Ratio,total_upt,120111.0
3078,The Eastern Contra Costa Transit Authority,2019,90162,Full Reporter,4,DR,PT,Urban,Group C,Farebox Recovery Ratio,total_vrh,43952.0
5538,The Eastern Contra Costa Transit Authority,2019,90162,Full Reporter,4,DR,PT,Urban,Group C,Farebox Recovery Ratio,total_pmt,703294.0
616,The Eastern Contra Costa Transit Authority,2019,90162,Full Reporter,4,DR,TN,Urban,Group C,Farebox Recovery Ratio,total_upt,NaN
3076,The Eastern Contra Costa Transit Authority,2019,90162,Full Reporter,4,DR,TN,Urban,Group C,Farebox Recovery Ratio,total_vrh,NaN
5536,The Eastern Contra Costa Transit Authority,2019,90162,Full Reporter,4,DR,TN,Urban,Group C,Farebox Recovery Ratio,total_pmt,NaN
617,The Eastern Contra Costa Transit Authority,2019,90162,Full Reporter,4,DR,TX,Urban,Group C,Farebox Recovery Ratio,total_upt,40235.0
3077,The Eastern Contra Costa Transit Authority,2019,90162,Full Reporter,4,DR,TX,Urban,Group C,Farebox Recovery Ratio,total_vrh,9919.0
5537,The Eastern Contra Costa Transit Authority,2019,90162,Full Reporter,4,DR,TX,Urban,Group C,Farebox Recovery Ratio,total_pmt,280052.0
614,The Eastern Contra Costa Transit Authority,2019,90162,Full Reporter,4,MB,PT,Urban,Group C,Farebox Recovery Ratio,total_upt,1825574.0


In [112]:
melt_funding[melt_funding["ntd_id"].isin([sample_ids[1]])].sort_values(by=["year","mode","type_of_service"])

,source_agency,year,ntd_id,caltrans_district,mode,type_of_service,urban_rural,cohort,metric,ntd_metric,ntd_metric_value
593,The Eastern Contra Costa Transit Authority,2019,90162,4,DR,PT,Urban,Group C,Local Funding % Change vs 2019,total_upt,120111.0
3098,The Eastern Contra Costa Transit Authority,2019,90162,4,DR,PT,Urban,Group C,Local Funding % Change vs 2019,total_vrh,43952.0
5603,The Eastern Contra Costa Transit Authority,2019,90162,4,DR,PT,Urban,Group C,Local Funding % Change vs 2019,total_pmt,703294.0
591,The Eastern Contra Costa Transit Authority,2019,90162,4,DR,TN,Urban,Group C,Local Funding % Change vs 2019,total_upt,NaN
3096,The Eastern Contra Costa Transit Authority,2019,90162,4,DR,TN,Urban,Group C,Local Funding % Change vs 2019,total_vrh,NaN
5601,The Eastern Contra Costa Transit Authority,2019,90162,4,DR,TN,Urban,Group C,Local Funding % Change vs 2019,total_pmt,NaN
592,The Eastern Contra Costa Transit Authority,2019,90162,4,DR,TX,Urban,Group C,Local Funding % Change vs 2019,total_upt,40235.0
3097,The Eastern Contra Costa Transit Authority,2019,90162,4,DR,TX,Urban,Group C,Local Funding % Change vs 2019,total_vrh,9919.0
5602,The Eastern Contra Costa Transit Authority,2019,90162,4,DR,TX,Urban,Group C,Local Funding % Change vs 2019,total_pmt,280052.0
589,The Eastern Contra Costa Transit Authority,2019,90162,4,MB,PT,Urban,Group C,Local Funding % Change vs 2019,total_upt,1825574.0


## aggregation group by
- farebox melt
    - PMT, UPT, VRH totals for urban, per year
    - PMT, UPT, VRH totals for rural, per year
    - PMT, UPT, VRH totals for cohort A, per year
    - PMT, UPT, VRH totals for cohort B, per year
    - PMT, UPT, VRH totals for cohort C, per year
- funding melt
    - PMT, UPT, VRH totals for urban, per year
    - PMT, UPT, VRH totals for rural, per year
    - PMT, UPT, VRH totals for cohort A, per year
    - PMT, UPT, VRH totals for cohort B, per year
    - PMT, UPT, VRH totals for cohort C, per year

In [149]:
group_list_agg = [
    "source_agency",
    "year",
    "ntd_id",
    "caltrans_district",
    
]
farebox_vrh_total = (
    melt_farebox[melt_farebox["ntd_metric"] == "total_vrh"]
    .groupby(group_list_agg)["ntd_metric_value"]
    .sum()
    .reset_index()
).rename(columns={"ntd_metric_value": "total_vrh"})

farebox_upt_total = (
    melt_farebox[melt_farebox["ntd_metric"] == "total_upt"]
    .groupby(group_list_agg)["ntd_metric_value"]
    .sum()
    .reset_index()
).rename(columns={"ntd_metric_value": "total_upt"})

farebox_pmt_total = (
    melt_farebox[melt_farebox["ntd_metric"] == "total_pmt"]
    .groupby(group_list_agg)["ntd_metric_value"]
    .sum()
    .reset_index()
).rename(columns={"ntd_metric_value": "total_pmt"})

farebox_cohort_totals = (
    cohort_merge_farebox.groupby(["year","cohort"])
    .agg({"total_upt": "sum", "total_vrh": "sum", "total_pmt": "sum"})
    .reset_index()
)

farebox_area_totals = (
    cohort_merge_farebox.groupby(["year","urban_rural"])
    .agg({"total_upt": "sum", "total_vrh": "sum", "total_pmt": "sum"})
    .reset_index()
)

In [152]:
funding_vrh_total = (
    melt_funding[melt_funding["ntd_metric"] == "total_vrh"]
    .groupby(group_list_agg)["ntd_metric_value"]
    .sum()
    .reset_index()
).rename(columns={"ntd_metric_value": "total_vrh"})

funding_upt_total = (
    melt_funding[melt_funding["ntd_metric"] == "total_upt"]
    .groupby(group_list_agg)["ntd_metric_value"]
    .sum()
    .reset_index()
).rename(columns={"ntd_metric_value": "total_upt"})

funding_pmt_total = (
    melt_funding[melt_funding["ntd_metric"] == "total_pmt"]
    .groupby(group_list_agg)["ntd_metric_value"]
    .sum()
    .reset_index()
).rename(columns={"ntd_metric_value": "total_pmt"})

funding_cohort_totals = (
    cohort_merge_funding.groupby(["year","cohort"])
    .agg({"total_upt": "sum", "total_vrh": "sum", "total_pmt": "sum"})
    .reset_index()
)

funding_area_totals = (
    cohort_merge_funding.groupby(["year","urban_rural"])
    .agg({"total_upt": "sum", "total_vrh": "sum", "total_pmt": "sum"})
    .reset_index()
)

## summary stats

In [153]:
metric_df = [
    farebox_upt_total, 
    farebox_pmt_total, 
    farebox_vrh_total,
    funding_upt_total, 
    funding_pmt_total, 
    funding_vrh_total,
]

for df in metric_df:
    display(
        df.drop(columns="year").describe()
    )

,total_upt
count,8.600000e+02
mean,6.879665e+06
std,3.265696e+07
min,0.000000e+00
25%,5.005375e+04
50%,2.779145e+05
75%,1.856973e+06
max,3.797181e+08


,total_pmt
count,8.600000e+02
mean,3.769781e+07
std,1.698335e+08
min,0.000000e+00
25%,0.000000e+00
50%,3.308975e+05
75%,1.063338e+07
max,1.962038e+09


,total_vrh
count,8.600000e+02
mean,2.965975e+05
std,1.012000e+06
min,0.000000e+00
25%,1.133375e+04
50%,3.907050e+04
75%,1.536380e+05
max,8.843739e+06


,total_upt
count,9.010000e+02
mean,5.675725e+06
std,2.802095e+07
min,0.000000e+00
25%,3.991600e+04
50%,2.370610e+05
75%,1.460752e+06
max,3.797181e+08


,total_pmt
count,9.010000e+02
mean,3.117503e+07
std,1.443151e+08
min,0.000000e+00
25%,0.000000e+00
50%,0.000000e+00
75%,9.638938e+06
max,1.962038e+09


,total_vrh
count,9.010000e+02
mean,2.476097e+05
std,7.993872e+05
min,0.000000e+00
25%,9.407000e+03
50%,3.523600e+04
75%,1.388800e+05
max,8.843739e+06


In [162]:
farebox_cohort_totals

,year,cohort,total_upt,total_vrh,total_pmt
0,2019,Group A,481794553.0,16706633.0,4.436104e+09
1,2019,Group B,912340315.0,28495217.0,3.939125e+09
2,2019,Group C,20087538.0,1850617.0,9.248414e+07
3,2020,Group A,352277028.0,13668740.0,3.282966e+09
4,2020,Group B,742982204.0,27311983.0,3.208062e+09
5,2020,Group C,29816757.0,2235922.0,1.039316e+08
6,2021,Group A,232748151.0,15751885.0,1.143152e+09
7,2021,Group B,305515511.0,18526592.0,1.521518e+09
8,2021,Group C,21506525.0,2299048.0,6.720324e+07
9,2022,Group A,339112596.0,17344827.0,1.940673e+09


In [184]:
pd.options.display.float_format = '{:,.2f}'.format

In [185]:
cohorts = ["Group A","Group B","Group C"]
ntd_metrics = ["total_upt", "total_vrh", "total_pmt"]
for cohort in cohorts:
  for metric in ntd_metrics:
    display(
        f"{cohort} cohort: Median and Mean {metric}",
        farebox_cohort_totals[farebox_cohort_totals["cohort"]==cohort][metric].describe(),
    )

'Group A cohort: Median and Mean total_upt'

count             6.00
mean    363,045,501.17
std      81,533,848.09
min     232,748,151.00
25%     342,403,704.00
50%     362,136,328.00
75%     393,257,695.25
max     481,794,553.00
Name: total_upt, dtype: float64

'Group A cohort: Median and Mean total_vrh'

count            6.00
mean    15,813,005.67
std      1,495,824.20
min     13,668,740.00
25%     14,734,940.50
50%     16,229,259.00
75%     16,934,150.75
max     17,344,827.00
Name: total_vrh, dtype: float64

'Group A cohort: Median and Mean total_pmt'

...

count             6.00
mean     67,954,291.33
std     101,615,457.25
min      20,087,538.00
25%      23,180,320.50
50%      29,009,232.00
75%      32,190,993.00
max     275,130,816.00
Name: total_upt, dtype: float64

'Group C cohort: Median and Mean total_vrh'

count           6.00
mean    3,466,684.50
std     2,982,839.26
min     1,850,617.00
25%     2,251,703.50
50%     2,299,923.50
75%     2,507,914.50
max     9,536,768.00
Name: total_vrh, dtype: float64

'Group C cohort: Median and Mean total_pmt'

count               6.00
mean      252,554,557.83
std       394,293,811.00
min        67,203,239.00
25%        86,459,505.25
50%        98,207,870.00
75%       108,821,553.00
max     1,056,805,537.00
Name: total_pmt, dtype: float64

In [186]:
display(
    farebox_cohort_totals,
    funding_cohort_totals,
)

,year,cohort,total_upt,total_vrh,total_pmt
0,2019,Group A,"481,794,553.00","16,706,633.00","4,436,104,017.00"
1,2019,Group B,"912,340,315.00","28,495,217.00","3,939,125,127.00"
2,2019,Group C,"20,087,538.00","1,850,617.00","92,484,145.00"
3,2020,Group A,"352,277,028.00","13,668,740.00","3,282,966,486.00"
4,2020,Group B,"742,982,204.00","27,311,983.00","3,208,061,903.00"
5,2020,Group C,"29,816,757.00","2,235,922.00","103,931,595.00"
6,2021,Group A,"232,748,151.00","15,751,885.00","1,143,151,643.00"
7,2021,Group B,"305,515,511.00","18,526,592.00","1,521,517,535.00"
8,2021,Group C,"21,506,525.00","2,299,048.00","67,203,239.00"
9,2022,Group A,"339,112,596.00","17,344,827.00","1,940,673,002.00"


,year,cohort,total_upt,total_vrh,total_pmt
0,2019,Group A,"148,588,010.00","8,696,556.00","1,008,222,676.00"
1,2019,Group B,"1,013,268,987.00","27,355,817.00","6,193,402,337.00"
2,2019,Group C,"78,329,932.00","5,536,365.00","422,818,499.00"
3,2020,Group A,"99,043,391.00","6,192,452.00","546,200,398.00"
4,2020,Group B,"454,096,208.00","19,486,133.00","2,408,906,495.00"
5,2020,Group C,"429,594,138.00","12,507,084.00","2,925,227,657.00"
6,2021,Group A,"77,094,314.00","7,204,425.00","493,327,080.00"
7,2021,Group B,"295,977,041.00","18,366,058.00","1,349,474,466.00"
8,2021,Group C,"109,997,160.00","6,162,919.00","466,817,866.00"
9,2022,Group A,"74,711,924.00","6,127,962.00","418,870,194.00"


In [187]:
display(
    farebox_area_totals,
    funding_area_totals
)

,year,urban_rural,total_upt,total_vrh,total_pmt
0,2019,Rural,"5,928,494.00","834,029.00",0.00
1,2019,Urban,"1,408,293,912.00","46,218,438.00","8,467,713,289.00"
2,2020,Rural,"4,798,067.00","770,218.00",0.00
3,2020,Urban,"1,120,277,922.00","42,446,427.00","6,594,959,984.00"
4,2021,Rural,"2,326,733.00","646,031.00",0.00
5,2021,Urban,"557,443,454.00","35,931,494.00","2,731,872,417.00"
6,2022,Rural,"3,535,335.00","678,449.00",0.00
7,2022,Urban,"800,388,439.00","40,185,140.00","4,104,801,024.00"
8,2023,Rural,"4,226,671.00","712,702.00",0.00
9,2023,Urban,"948,815,353.00","42,058,866.00","4,962,163,814.00"


,year,urban_rural,total_upt,total_vrh,total_pmt
0,2019,Rural,"5,903,838.00","827,864.00",0.00
1,2019,Urban,"1,234,283,091.00","40,760,874.00","7,624,443,512.00"
2,2020,Rural,"4,824,834.00","776,637.00",0.00
3,2020,Urban,"977,908,903.00","37,409,032.00","5,880,334,550.00"
4,2021,Rural,"2,370,192.00","662,767.00",0.00
5,2021,Urban,"480,698,323.00","31,070,635.00","2,309,619,412.00"
6,2022,Rural,"3,700,019.00","723,578.00",0.00
7,2022,Urban,"683,949,382.00","34,664,550.00","3,442,444,852.00"
8,2023,Rural,"4,374,666.00","747,190.00",0.00
9,2023,Urban,"809,475,174.00","36,582,016.00","4,178,216,536.00"
